<a href="https://colab.research.google.com/github/Pravinoraon/capstone/blob/main/Notebook%202%3A%20Model%20Training%20with%20ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras==2.14.0
!pip install vit-keras
!pip install tensorflow==2.15.0 keras==2.15.0


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.applications import ResNet50, MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from vit_keras import vit
from google.colab import drive

ModuleNotFoundError: No module named 'vit_keras'

In [ ]:
# Mount Drive
drive.mount('/content/drive')

In [ ]:
# Define dataset paths
train_dir = "/content/drive/MyDrive/bell_pepper_data/train"
val_dir = "/content/drive/MyDrive/bell_pepper_data/val"

In [ ]:
# Verify dataset paths
print("Train directory exists:", os.path.exists(train_dir))
print("Validation directory exists:", os.path.exists(val_dir))
print("Train categories:", os.listdir(train_dir) if os.path.exists(train_dir) else "Not found")
print("Validation categories:", os.listdir(val_dir) if os.path.exists(val_dir) else "Not found")


In [ ]:
# Define constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Create train & validation generators
train_data = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary'
)

# Define input layer
inputs = Input(shape=(224, 224, 3))


In [ ]:
# Load ResNet50
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
resnet.trainable = False
resnet_features = GlobalAveragePooling2D()(resnet(inputs))

# Load MobileNetV2
mobilenet = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
mobilenet.trainable = False
mobilenet_features = GlobalAveragePooling2D()(mobilenet(inputs))

# Load Vision Transformer (ViT)
vit_model = vit.vit_b16(
    image_size=224,
    activation='softmax',
    pretrained=True,
    include_top=False,
    pretrained_top=False
)
vit_features = vit_model(inputs)

In [ ]:
# Combine features from all three models
merged_features = Concatenate()([resnet_features, mobilenet_features, vit_features])

# Fully connected layers
x = Dense(256, activation="relu")(merged_features)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.2)(x)

# Output layer for binary classification
outputs = Dense(1, activation="sigmoid", dtype=tf.float32)(x)

# Define final model
model = Model(inputs, outputs)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Model Summary
model.summary()

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[early_stopping]
)

In [ ]:
# Save trained model
model_save_path = "/content/drive/MyDrive/bell_pepper_data/bell_pepper_disease_model_hybrid.h5"
model.save(model_save_path)
print(f"Model saved at: {model_save_path}")

# Plot training & validation accuracy/loss
plt.figure(figsize=(12, 5))


In [ ]:
# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()
